<a href="https://colab.research.google.com/github/FarhinaMirza/CE888/blob/main/lab3_End_to_End_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

In [8]:
from google.colab import files
uploaded = files.upload()

Saving bank-additional-full.csv to bank-additional-full (1).csv


In [9]:
df = pd.read_csv("bank-additional-full.csv", sep=';')

In [10]:
#df=pd.read_csv('/content/drive/MyDrive/CE888/Lab 3/bank-additional/bank-additional-full.csv',sep=';')

In [11]:
df.shape

(41188, 21)

In [12]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [13]:
df.drop('duration', inplace=True, axis=1)

In [14]:
df.shape

(41188, 20)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  campaign        41188 non-null  int64  
 11  pdays           41188 non-null  int64  
 12  previous        41188 non-null  int64  
 13  poutcome        41188 non-null  object 
 14  emp.var.rate    41188 non-null  float64
 15  cons.price.idx  41188 non-null  float64
 16  cons.conf.idx   41188 non-null  float64
 17  euribor3m       41188 non-null 

In [16]:
# Check class imbalance 
df['y'].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

In [17]:
le1 = LabelEncoder()
df['label'] = le1.fit_transform(df['y'])
df.drop('y', inplace=True, axis=1)

In [18]:
cat_features = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome"]
df[cat_features].head()

# Question 3 : How many categorical features does the Bank dataset have?
#. Answer : 10

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome
0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,nonexistent
1,services,married,high.school,unknown,no,no,telephone,may,mon,nonexistent
2,services,married,high.school,no,yes,no,telephone,may,mon,nonexistent
3,admin.,married,basic.6y,no,no,no,telephone,may,mon,nonexistent
4,services,married,high.school,no,no,yes,telephone,may,mon,nonexistent


In [19]:
# Let's also look at standardising some of the numerical features
num_features = ['age', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'  ]
df[num_features].head()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,56,1,999,0,1.1,93.994,-36.4,4.857,5191.0
1,57,1,999,0,1.1,93.994,-36.4,4.857,5191.0
2,37,1,999,0,1.1,93.994,-36.4,4.857,5191.0
3,40,1,999,0,1.1,93.994,-36.4,4.857,5191.0
4,56,1,999,0,1.1,93.994,-36.4,4.857,5191.0


In [20]:
# Most algorithms cannot work with missing values. Three main ways to deal with them:
# 1. Drop rows
# 2. Drop features with NaNs
# 3. Fill missing values
# As we have very few missing values, let's drop the rows
print("Number of rows before dropping NaNs: %d" % len(df))
df = df.dropna()
print("Number of rows after dropping NaNs: %d" % len(df))
# Separate features from outcomes
df_y = df['label'].copy()
df_X = df.iloc[:,:-1]

Number of rows before dropping NaNs: 41188
Number of rows after dropping NaNs: 41188


In [21]:
class OneHotEncoderCategoricalFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, cat_features):
        self.cat_features = cat_features
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return pd.get_dummies(X, columns=self.cat_features)

In [22]:
# We can use a ColumnTransformer to transform only some types of columns in different ways
colTransformer = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(), cat_features)
    ],
    remainder='passthrough'  # i.e., leave the rest of the columns untouched -- otherwise, they're dropped
    )
X_processed = colTransformer.fit_transform(df_X)
X_processed.shape

(41188, 62)

In [23]:
X_train,X_test,y_train,y_test = train_test_split(X_processed,df_y,random_state=42,stratify=df_y,test_size=0.3)

In [24]:
dc = DummyClassifier()
dc.fit(X_train, y_train)

dc_y_pred = dc.predict(X_test)

In [25]:
rf = RandomForestClassifier(n_estimators = 100 , max_depth = 3)
rf.fit(X_train, y_train)

rf_y_pred = rf.predict(X_test)

In [26]:
dc_f1_score = f1_score(y_test,dc_y_pred)
rf_f1_score = f1_score(y_test,rf_y_pred)
print('Dummy Classifier:', dc_f1_score)
print('Random Forest:', rf_f1_score)

Dummy Classifier: 0.0
Random Forest: 0.2471229557843731


In [27]:

# Question 4: In the bank dataset, the dummy (stratified) classifier is better than a random forest classifier 
# with 100 estimators and a maximum depth of 3, according to the F1 score.
# Answer : False

# Question 5: In the bank dataset, a random forest classifier with 100 estimators and a maximum depth of 3 is better
# than a dummy (stratified) classifier, according to the F1 score.
# Answer : True

In [34]:
# Divide your training and test set using train_test_split and keeping your test set to 20% of the full dataset, using random_state=50 when calling the function.
# Perform 10-fold cross-validation using as a classifier a random forest with 100 estimators and a maximum depth of 8. 
# Report the average F1 score with two numbers after the floating point (e.g., 0.56)

cv = KFold(n_splits=10)
rf_model = RandomForestClassifier(n_estimators=100,max_depth=8,random_state=60)
f1_scores = cross_val_score(rf_model, X_processed, df_y, scoring='f1', cv=cv)
f1_scores

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.07569141, 0.13822285])

In [35]:
print(np.average(f1_scores))

0.02139142610191692


In [30]:
#Divide your training and test set using train_test_split and keeping your test set to 20% of the full dataset, using random_state=50 when calling the function.
#Perform 10-fold cross-validation using as a classifier a random forest with 200 estimators and a maximum depth of 8. Report the average F1 score 
#with two numbers after the floating point (e.g., 0.56)

In [31]:
cv = KFold(n_splits=10)
rf_model = RandomForestClassifier(n_estimators=200,max_depth=8,random_state=60)
f1_scores = cross_val_score(rf_model, X_processed, df_y, scoring='f1', cv=cv)
f1_scores

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.07580175, 0.13929412])

In [32]:
print(np.average(f1_scores))

0.021509586691819585


In [33]:
import pickle as pkl

outp = open('model.pkl', 'wb')
pkl.dump(clf, outp, -1)  # where clf is your classifier
outp.close()

NameError: ignored